In [1]:
import pandas as pd

patient = pd.read_csv("Z:\Ferretin Project\Ferritin Project\Patient.csv")
lab = pd.read_csv("Z:\Ferretin Project\Ferritin Project\Lab.csv")

# checking that inner join doesn't lose any patients.
lab_patient_merged = pd.merge(patient, lab, on = "Patient_ID", how = 'inner')
lab_patient_merged_outer = pd.merge(patient, lab, on = "Patient_ID", how = 'outer')

lab_patient_merged['Site_ID'] = lab_patient_merged['Site_ID_x'].combine_first(lab_patient_merged['Site_ID_y'])
lab_patient_merged.drop(['Site_ID_x', 'Site_ID_y'], axis=1, inplace=True)



C:\Users\suzielee\AppData\Local\Temp\ipykernel_9128\3479589883.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  lab = pd.read_csv("Z:\Ferretin Project\Ferritin Project\Lab.csv")


In [2]:
import numpy as np
lab_patient_merged['DateCreated'] = pd.to_datetime(lab_patient_merged['DateCreated'])

mask = ~lab_patient_merged[['DateCreated', 'BirthYear']].isnull().any(axis=1)
lab_patient_merged['Age'] = np.nan
lab_patient_merged.loc[mask, 'Age'] = (
    (lab_patient_merged.loc[mask, 'DateCreated'] - 
    pd.to_datetime(lab_patient_merged.loc[mask, 'BirthYear'].astype(str) + '-06-01'))
    .dt.days // 365
)

# Filter data for age >= 6
filtered_data = lab_patient_merged[lab_patient_merged['Age'] >= 6]
filtered_data['YearCreated'] = filtered_data['DateCreated'].dt.year.copy()



C:\Users\suzielee\AppData\Local\Temp\ipykernel_9128\3779377110.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  lab_patient_merged.loc[mask, 'Age'] = (
C:\Users\suzielee\AppData\Local\Temp\ipykernel_9128\3779377110.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['YearCreated'] = filtered_data['DateCreated'].dt.year.copy()


In [3]:
ferritin_lab = filtered_data[lab_patient_merged['Code_calc'] == "2276-4"]
ferritin_lab_short = ferritin_lab[['Patient_ID', 'Sex', 'Site_ID', 'TestResult_calc', 'YearCreated', 'Age']].copy()

C:\Users\suzielee\AppData\Local\Temp\ipykernel_9128\4070243153.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ferritin_lab = filtered_data[lab_patient_merged['Code_calc'] == "2276-4"]


In [4]:
#Iron Deficiency
ferritin_lab_short['TestResult_calc'] = pd.to_numeric(ferritin_lab_short['TestResult_calc'], errors='coerce')
ferritin_lab_short['iron_deficiency'] = ferritin_lab_short.groupby(['YearCreated', 'Patient_ID'])['TestResult_calc']\
                                        .transform(lambda x: (x < 15).any()).astype(int)


In [5]:
# Sorting such that rows with iron deficiency come on top. #Then deleting the duplicate rows to keep each unique patient only once
ferritin_lab_short_sorted = ferritin_lab_short.sort_values(by=['iron_deficiency'], ascending=[False])
ferritin_lab_short_unique = ferritin_lab_short_sorted.drop_duplicates(subset=['Patient_ID'])

total_patients_with_iron_deficiency = ferritin_lab_short_unique[ferritin_lab_short_unique['iron_deficiency'] == 1]['Patient_ID'].nunique()
total_patients_with_iron_sufficiency = ferritin_lab_short_unique[ferritin_lab_short_unique['iron_deficiency'] == 0]['Patient_ID'].nunique()
total_unique_patients = ferritin_lab_short_unique['Patient_ID'].nunique()
proportion_with_iron_deficiency = total_patients_with_iron_deficiency / total_unique_patients
proportion_with_iron_sufficiency = total_patients_with_iron_sufficiency / total_unique_patients

print("Total unique patients with Iron Deficiency:", total_patients_with_iron_deficiency)
print("Total unique patients with Iron Sufficiency:", total_patients_with_iron_sufficiency)
print("Proportion of patients with Iron Deficiency:", proportion_with_iron_deficiency)
print("Proportion of patients with Iron Sufficiency:", proportion_with_iron_sufficiency)


Total unique patients with Iron Deficiency: 10081
Total unique patients with Iron Sufficiency: 84183
Proportion of patients with Iron Deficiency: 0.10694432657218025
Proportion of patients with Iron Sufficiency: 0.8930556734278198


In [6]:
# Making sure #s are consistent. 
print(ferritin_lab_short_unique['Patient_ID'].nunique())

94264


In [7]:
#Counts by sex
total_unique_females = ferritin_lab_short_unique[ferritin_lab_short_unique['Sex'] == 'Female']['Patient_ID'].nunique()
total_unique_males = ferritin_lab_short_unique[ferritin_lab_short_unique['Sex'] == 'Male']['Patient_ID'].nunique()

#Patients with unspecified gender (NaN in 'Sex' column)
total_unique_other = ferritin_lab_short_unique['Patient_ID'].nunique() - total_unique_females - total_unique_males

#Tally for each sex category
count_females_with_iron_deficiency = ferritin_lab_short_unique[(ferritin_lab_short_unique['Sex'] == 'Female') & (ferritin_lab_short_unique['iron_deficiency'] == 1)]['Patient_ID'].nunique()
count_males_with_iron_deficiency = ferritin_lab_short_unique[(ferritin_lab_short_unique['Sex'] == 'Male') & (ferritin_lab_short_unique['iron_deficiency'] == 1)]['Patient_ID'].nunique()
count_other_with_iron_deficiency = ferritin_lab_short_unique[(ferritin_lab_short_unique['Sex'].isna()) & (ferritin_lab_short_unique['iron_deficiency'] == 1)]['Patient_ID'].nunique()
proportion_females_with_iron_deficiency = count_females_with_iron_deficiency / total_unique_females
proportion_males_with_iron_deficiency = count_males_with_iron_deficiency / total_unique_males
proportion_other_with_iron_deficiency = count_other_with_iron_deficiency / total_unique_other

print("\nCounts and Proportions by Sex:")
print("Total unique females:", total_unique_females)
print("Total unique males:", total_unique_males)
print("Total unique other/unspecified:", total_unique_other)
print("Count of females with Iron Deficiency:", count_females_with_iron_deficiency)
print("Count of males with Iron Deficiency:", count_males_with_iron_deficiency)
print("Count of other/unspecified with Iron Deficiency:", count_other_with_iron_deficiency)
print("Proportion of females with Iron Deficiency:", proportion_females_with_iron_deficiency)
print("Proportion of males with Iron Deficiency:", proportion_males_with_iron_deficiency)
print("Proportion of other/unspecified with Iron Deficiency:", proportion_other_with_iron_deficiency)



Counts and Proportions by Sex:
Total unique females: 58007
Total unique males: 36213
Total unique other/unspecified: 44
Count of females with Iron Deficiency: 9113
Count of males with Iron Deficiency: 964
Count of other/unspecified with Iron Deficiency: 4
Proportion of females with Iron Deficiency: 0.15710172910166015
Proportion of males with Iron Deficiency: 0.026620274487062657
Proportion of other/unspecified with Iron Deficiency: 0.09090909090909091


In [8]:
from scipy.stats import chi2_contingency

contingency_table_sex_counts = [
    [count_females_with_iron_deficiency, total_unique_females - count_females_with_iron_deficiency],
    [count_males_with_iron_deficiency, total_unique_males - count_males_with_iron_deficiency],
    [count_other_with_iron_deficiency, total_unique_other - count_other_with_iron_deficiency]
]

chi2_sex_counts, p_sex_counts, _, _ = chi2_contingency(contingency_table_sex_counts)

print("\nChi-Squared Test Results:")
print(f"Chi-Squared Statistic (Sex vs. Iron Deficiency Counts): {chi2_sex_counts:.10f}")
print(f"P-Value: {p_sex_counts:.10f}")


Chi-Squared Test Results:
Chi-Squared Statistic (Sex vs. Iron Deficiency Counts): 3974.4410129739
P-Value: 0.0000000000


In [9]:
#Double-checking manually with tally #s. 

total_females = 58007
total_males = 36213
total_others = 44

contingency_table_sex_counts = [[9113, total_females - 9113],
                                [964, total_males - 964],
                                [4, total_others - 4]]

chi2_sex_counts, p_sex_counts, _, _ = chi2_contingency(contingency_table_sex_counts)

print(f"Chi-Squared Statistic (Sex vs. Iron Deficiency Counts): {chi2_sex_counts}")
print(f"P-Value: {p_sex_counts:.10f}")

Chi-Squared Statistic (Sex vs. Iron Deficiency Counts): 3974.4410129738626
P-Value: 0.0000000000


In [10]:
def categorize_age_group(age):
    if 6 <= age <= 13:
        return '6-13'
    elif 14 <= age <= 18:
        return '14-18'
    elif 19 <= age <= 30:
        return '19-30'
    elif 31 <= age <= 50:
        return '31-50'
    elif 51 <= age <= 79:
        return '51-79'
    else:
        return '80+'

ferritin_lab_short_unique['AgeGroup'] = ferritin_lab_short_unique['Age'].apply(categorize_age_group)
age_categories = ['6-13', '14-18', '19-30', '31-50', '51-79', '80+']

# Dictionaries
count_by_age = {}
proportion_by_age = {}
total_unique_pts = {}

# Iterate for age categories
for age_group in age_categories:
    filtered_data = ferritin_lab_short_unique[ferritin_lab_short_unique['AgeGroup'] == age_group]
    count_with_iron_deficiency = filtered_data[filtered_data['iron_deficiency'] == 1]['Patient_ID'].nunique()
    total_unique_patients = filtered_data['Patient_ID'].nunique()
    proportion = count_with_iron_deficiency / total_unique_patients
    
    count_by_age[age_group] = count_with_iron_deficiency
    proportion_by_age[age_group] = proportion
    total_unique_pts[age_group] = total_unique_patients

print("Counts by Age Group:", count_by_age)
print(count_by_age)
print("\nProportions by Age Group:")
print(proportion_by_age)
print("\nOverall population by Age Group:")
print(total_unique_pts)



C:\Users\suzielee\AppData\Local\Temp\ipykernel_9128\3621126437.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ferritin_lab_short_unique['AgeGroup'] = ferritin_lab_short_unique['Age'].apply(categorize_age_group)


Counts by Age Group: {'6-13': 249, '14-18': 663, '19-30': 2005, '31-50': 4683, '51-79': 2173, '80+': 308}
{'6-13': 249, '14-18': 663, '19-30': 2005, '31-50': 4683, '51-79': 2173, '80+': 308}

Proportions by Age Group:
{'6-13': 0.053387650085763294, '14-18': 0.13916876574307305, '19-30': 0.14294880935405674, '31-50': 0.14515979045906824, '51-79': 0.06332876752251333, '80+': 0.07271010387157696}

Overall population by Age Group:
{'6-13': 4664, '14-18': 4764, '19-30': 14026, '31-50': 32261, '51-79': 34313, '80+': 4236}


In [11]:
total_age_categories = [total_unique_pts[age_group] for age_group in age_categories]
iron_deficiency_cases_age = [count_by_age[age_group] for age_group in age_categories]
non_deficiency_cases_age = [total - deficient for total, deficient in zip(total_age_categories, iron_deficiency_cases_age)]

contingency_table_age_counts = [[deficient, non_deficient] for deficient, non_deficient in zip(iron_deficiency_cases_age, non_deficiency_cases_age)]
chi2_age_counts, p_age_counts, _, _ = chi2_contingency(contingency_table_age_counts)

print(f"Chi-squared test: age vs iron Deficiency): {chi2_age_counts}")
print(f"P-Value: {p_age_counts:.4f}")  


Chi-squared test: age vs iron Deficiency): 1610.9820529523645
P-Value: 0.0000


In [12]:
# Check manually
total_age_categories = [4664, 4764, 14026, 32261, 34313, 4236]
iron_deficiency_cases_age = [249, 663, 2005, 4683, 2173, 308]
non_deficiency_cases_age = [4415, 4101, 12021, 27578, 32140, 3928]
contingency_table_age_counts = [[deficient, non_deficient] for deficient, non_deficient in zip(iron_deficiency_cases_age, non_deficiency_cases_age)]
chi2_age_counts, p_age_counts, _, _ = chi2_contingency(contingency_table_age_counts)

print(f"Chi=Squared test: age vs iron Deficiency): {chi2_age_counts}")
print(f"P-Value: {p_age_counts:.3f}")

Chi=Squared test: age vs iron Deficiency): 1610.9820529523645
P-Value: 0.000


In [13]:
depr = pd.read_csv("Z:\Ferretin Project\Ferritin Project\Deprivation.csv")

depr['mat1'] = (1*depr['MaterialQ1'])/100
depr['mat2'] = (2 * depr['MaterialQ2']) / 100
depr['mat3'] = (3 * depr['MaterialQ3']) / 100
depr['mat4'] = (4 * depr['MaterialQ4']) / 100
depr['mat5'] = (5 * depr['MaterialQ5']) / 100

depr['matdeptotal'] = depr[['mat1', 'mat2', 'mat3', 'mat4', 'mat5']].sum(axis=1)

ferritin_mat = pd.merge(ferritin_lab_short_unique, depr[['Patient_ID', 'matdeptotal']], on="Patient_ID", how='left')

total_patients_with_iron_deficiency = ferritin_lab_short_unique[ferritin_lab_short_unique['iron_deficiency'] == 1]['Patient_ID'].nunique()
total_patients_with_iron_sufficiency = ferritin_lab_short_unique[ferritin_lab_short_unique['iron_deficiency'] == 0]['Patient_ID'].nunique()
print(total_patients_with_iron_deficiency, total_patients_with_iron_sufficiency)

10081 84183


In [14]:
count_by_category = {}
proportion_by_category = {}
total_unique_patients = {} 

categories = ['0-1', '1-2', '2-3', '3-4', '4-5']

# Function to categorize matdeptotal
def categorize_matdeptotal(matdeptotal):
    if 0 <= matdeptotal <= 1:
        return '0-1'
    elif 1 < matdeptotal <= 2:
        return '1-2'
    elif 2 < matdeptotal <= 3:
        return '2-3'
    elif 3 < matdeptotal <= 4:
        return '3-4'
    else:
        return '4-5'

ferritin_mat['MatDeptotalCategory'] = ferritin_mat['matdeptotal'].apply(categorize_matdeptotal)

# Iterate through matdeptotal
for category in categories:
    filtered_data = ferritin_mat[ferritin_mat['MatDeptotalCategory'] == category]
    
    count_with_iron_deficiency = filtered_data[filtered_data['iron_deficiency'] == 1]['Patient_ID'].nunique()
    
    total_unique_patients_in_category = filtered_data['Patient_ID'].nunique()
    
    # Storage
    count_by_category[category] = count_with_iron_deficiency
    proportion_by_category[category] = proportion
    total_unique_patients[category] = total_unique_patients_in_category

print("Counts by MatDep:")
print(count_by_category)
print("\nProportions by MatDep:")
print(proportion_by_category)
print("\nOverall population by MatDep:")
print(total_unique_patients)


Counts by MatDep:
{'0-1': 2662, '1-2': 1661, '2-3': 1475, '3-4': 1997, '4-5': 2286}

Proportions by MatDep:
{'0-1': 0.07271010387157696, '1-2': 0.07271010387157696, '2-3': 0.07271010387157696, '3-4': 0.07271010387157696, '4-5': 0.07271010387157696}

Overall population by MatDep:
{'0-1': 29057, '1-2': 17110, '2-3': 13686, '3-4': 15260, '4-5': 19151}


In [15]:
# Using list
total_counts = list(total_unique_patients.values()) 
iron_deficiency_cases = list(count_by_category.values())

non_deficiency_cases = [total - deficient for total, deficient in zip(total_counts, iron_deficiency_cases)]
contingency_table = [[deficient, non_deficient] for deficient, non_deficient in zip(iron_deficiency_cases, non_deficiency_cases)]

chi2_statistic, p_value, _, _ = chi2_contingency(contingency_table)

print(f"Chi-Square independence: MatDep vs Iron Deficiency Counts): {chi2_statistic}")
print(f"P-Value: {p_value:.4f}")

Chi-Square independence: MatDep vs Iron Deficiency Counts): 211.42001278536017
P-Value: 0.0000


In [16]:
from platform import python_version

print(python_version())

3.11.4
